In [38]:
from DNASkittleUtils.Contigs import *
from DNASkittleUtils.DDVUtils import pp
from shutil import copy

In [3]:
contigs = read_contigs(r"D:\josiah\Documents\Research\Thesis - Genome Symmetry\DNA_Duplications\Green_Ash_Annotation_Transfer\fraxinus_pennsylvanica_20Feb2018_lPmM4_psuedochromosomes__gold_standard.fasta")

In [4]:
other_contigs = read_contigs(r"D:\josiah\Documents\Research\Thesis - Genome Symmetry\DNA_Duplications\Green_Ash_Annotation_Transfer\fraxinus_pennsylvanica_20Feb2018_lPmM4_psuedochromosomes.fasta")

In [5]:
len(contigs), len(other_contigs)

(23, 23)

In [70]:
paired = zip(contigs, other_contigs)
# [(len(a.seq), len(b.seq)) for a,b in paired]
[(b.name, len(b.seq)) for a,b in paired]

[('Chr01', 56547140),
 ('Chr02', 44747177),
 ('Chr03', 35495138),
 ('Chr04', 37650951),
 ('Chr05', 40543111),
 ('Chr06', 33221578),
 ('Chr07', 30837396),
 ('Chr08', 31844244),
 ('Chr09', 34315355),
 ('Chr10', 33661437),
 ('Chr11', 30912502),
 ('Chr12', 35280915),
 ('Chr13', 35000339),
 ('Chr14', 27471413),
 ('Chr15', 32063413),
 ('Chr16', 26444360),
 ('Chr17', 33065801),
 ('Chr18', 27868735),
 ('Chr19', 25977958),
 ('Chr20', 22152616),
 ('Chr21', 27325502),
 ('Chr22', 25839949),
 ('Chr23', 26798730)]

In [6]:
source = read_contigs("fraxinus_pennsylvanica_20Feb2018_lPmM4.fasta")

In [7]:
def list_to_dict(contigs):
    return {contig.name: contig for contig in contigs}
source = list_to_dict(source)

In [12]:
len(source["Scaffold_100000;HRSCAF=100831"].seq)

376

In [13]:
scaffolds = {key.split(';')[0]: value for key, value in source.items()}

In [14]:
scaffolds['Scaffold_243852']

< "Scaffold_243852;HRSCAF=245055" 9450333 nucleotides>

In [15]:
scaffolds['Scaffold_1']

< "Scaffold_1;HRSCAF=23" 61578 nucleotides>

In [13]:
pp(20209485 - 18981584)

'1,227,901'

# Removing HRSCAF from scaffold names to match annotation

# [Flo Documentation](https://github.com/wurmlab/flo)
A common problem encountered is that 1st column of GFF file doesn't match chromosome, or scaffold, or contig id in the source assembly. In this case liftOver will generate an empty output file. flo stops at this point. You can fix the GFF file and resume flo by running the above command.

In [14]:
anno_names = set()
with open('scrubbed_BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL.gff', 'r') as gff:
    lines = [line for line in gff.readlines() if not line.startswith('#')]
    for line in lines:
        cols = line.split('\t')
        anno_names.add(cols[0])
len(anno_names)

1048

In [15]:
[name for name in anno_names if name in source]

[]

In [16]:
list(source.keys())[:20]

['Scaffold_184327;HRSCAF=185158',
 'Scaffold_138191;HRSCAF=139022',
 'Scaffold_176308;HRSCAF=177139',
 'Scaffold_227188;HRSCAF=228019',
 'Scaffold_14934;HRSCAF=15765',
 'Scaffold_95779;HRSCAF=96610',
 'Scaffold_188136;HRSCAF=188967',
 'Scaffold_200501;HRSCAF=201332',
 'Scaffold_156644;HRSCAF=157475',
 'Scaffold_116445;HRSCAF=117276',
 'Scaffold_23491;HRSCAF=24322',
 'Scaffold_139451;HRSCAF=140282',
 'Scaffold_165015;HRSCAF=165846',
 'Scaffold_165542;HRSCAF=166373',
 'Scaffold_88360;HRSCAF=89191',
 'Scaffold_65459;HRSCAF=66290',
 'Scaffold_168971;HRSCAF=169802',
 'Scaffold_67624;HRSCAF=68455',
 'Scaffold_92141;HRSCAF=92972',
 'Scaffold_236355;HRSCAF=237186']

**Conclusion** HRSCAF needs to be scrubbed from the FASTA because it doesn't match the annotation

In [17]:
assert all([';' in contig.name for contig in source.values()])

In [18]:
for value in source.values():
    value.name = value.name.split(';')[0]

In [30]:
[contig.name for contig in source.values()][:20]

['Scaffold_240933',
 'Scaffold_71857',
 'Scaffold_210416',
 'Scaffold_192701',
 'Scaffold_86126',
 'Scaffold_135768',
 'Scaffold_13552',
 'Scaffold_241701',
 'Scaffold_216990',
 'Scaffold_53979',
 'Scaffold_162245',
 'Scaffold_25732',
 'Scaffold_81533',
 'Scaffold_156297',
 'Scaffold_189298',
 'Scaffold_60789',
 'Scaffold_92615',
 'Scaffold_117084',
 'Scaffold_182084',
 'Scaffold_5289']

In [21]:
write_contigs_to_file("fraxinus_pennsylvanica_20Feb2018_lPmM4_no_HRSCAF.fa", source.values())

Done writing  243852 contigs and 848,973,687bp


# Combining lifted and unlifted from non-chromosome scaffolds

In [ ]:
mapped = set('Scaffold_6 Scaffold_243849 Scaffold_222 Scaffold_243850 Scaffold_243841 Scaffold_132 Scaffold_243846 Scaffold_84 Scaffold_243835 Scaffold_243844 Scaffold_243845 Scaffold_243837 Scaffold_243847 Scaffold_243843 Scaffold_243848 Scaffold_243852 Scaffold_243842 Scaffold_243851 Scaffold_695 Scaffold_243839 Scaffold_509 Scaffold_122 Scaffold_429 Scaffold_243836 Scaffold_19 Scaffold_2 Scaffold_3668 Scaffold_2026 Scaffold_291'.split())

### hasevidenceMANUAL

In [87]:
orphans = [line.split('\t') for line in lines]
trouble = [orphan for orphan in orphans if orphan[0] in mapped]
len(trouble)

181

In [51]:
from os.path import dirname, join

In [90]:
def check_all_mains_were_lifted(lifted_gff):
    with open(lifted_gff, 'r') as infile:
        mains = set([line.split('\t')[0] for line in infile.readlines() if line.startswith('Chr')])
    assert len(mains) == 23, len(mains)

In [84]:
def combine_lifted_and_non(lifted, unlifted):
    check_all_mains_were_lifted(lifted_gff=lifted)
    destination = join(dirname(lifted), 'combined.gff3')
    copy(lifted, destination)
    with open(unlifted, 'r') as infile:
        lines = [line for line in infile.readlines() if not line.startswith('#')]
    
    with open(destination, 'a') as outfile:
        outfile.write('\n#Annotations in non-chromosome scaffolds that have not changed\n')
        outliers = [line for line in lines if line.split('\t')[0] not in mapped]
        outfile.writelines(outliers)
        print("Output", len(outliers), "extra lines")
        discarded = [line for line in lines if line.split('\t')[0] in mapped]
        print("Discarded", len(discarded), "lines from chromosomes")
    return discarded
discarded = combine_lifted_and_non(
lifted = r"flo_pennsylvanica\run_simplified\scrubbed_BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL\lifted.gff3",
unlifted = r"flo_pennsylvanica\run_simplified\scrubbed_BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL\unlifted.gff3")

Output 4990 extra lines
Discarded 0 lines from chromosomes


### Difference between lifted and lifted_cleaned?
Difference is that lifted.gff3 include useful comments from the original

### RepeatMasker Orphans

In [85]:
lifted = r"flo_pennsylvanica\run_simplified\fraxinus_pennsylvanica_20Feb2018_repeatmasker.fasta.out\lifted.gff3"
unlifted = r"flo_pennsylvanica\run_simplified\fraxinus_pennsylvanica_20Feb2018_repeatmasker.fasta.out\unlifted.gff3"
combine_lifted_and_non(lifted, unlifted)

Output 328552 extra lines
Discarded 0 lines from chromosomes


[]

In [63]:
def lines_without_comments(filepath):
    with open(filepath, 'r') as infile:
        lines = [line for line in infile.readlines() if not line.startswith('#')]
    return lines
lines = lines_without_comments(unlifted)
len(lines)

328733

In [88]:
orphans = [line.split('\t') for line in lines]
trouble = [orphan for orphan in orphans if orphan[0] in mapped]
len(trouble)

181

In [35]:
[' '.join(orphan) for orphan in orphans if 'Scaffold_243850' in orphan[0]]

['Scaffold_243850 RepeatMasker similarity 8350781 8350926 16.4 + . Target "Motif:rnd-2_family-180" 2 198\n',
 'Scaffold_243850 RepeatMasker similarity 9355883 9356069 5.3 - . Target "Motif:454CL159Contig64" 168 354\n']

Essentially all unlifted repeats are on non-chromosomes

All 23 chromosomes are represented in the lifted set

### No evidence MANUAL

In [91]:
combine_lifted_and_non(r"flo_pennsylvanica\run_simplified\scrubbed_BRAKER_RNASeqandEA_augustus_060618.hints.noevidenceMANUAL\lifted.gff3",
                      r"flo_pennsylvanica\run_simplified\scrubbed_BRAKER_RNASeqandEA_augustus_060618.hints.noevidenceMANUAL\unlifted.gff3")

Output 45446 extra lines
Discarded 0 lines from chromosomes


[]

### Coy down combined annotations with their folder names

In [92]:
copy(r"flo_pennsylvanica\run_simplified\scrubbed_BRAKER_RNASeqandEA_augustus_060618.hints.noevidenceMANUAL\combined.gff3",
r"final_versions/F_pennsylvanica_chr_BRAKER_RNASeqandEA_augustus_060618.hints.noevidenceMANUAL.gff3")

copy(r"flo_pennsylvanica\run_simplified\fraxinus_pennsylvanica_20Feb2018_repeatmasker.fasta.out\combined.gff3",
r"final_versions/F_pennsylvanica_chr_20Feb2018_repeatmasker.gff3")

copy(r"flo_pennsylvanica\run_simplified\scrubbed_BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL\combined.gff3",
r"final_versions/F_pennsylvanica_chr_BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL.gff3")

'final_versions/F_pennsylvanica_chr_BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL.gff3'

# Quality checking results against original inputs

**We would like to see roughly the same number of mRNA transcript entries in both input and output.  The should also have the same internal volume.**

In [93]:
def line_filter(lines, key):
    return [l for l in lines if key in l]

def quantify_transfer_percentage(transferred_file, original, key):
    original_lines = lines_without_comments(original)
    transfer_lines = lines_without_comments(transferred_file)
    t = len(line_filter(transfer_lines, key))
    o = len(line_filter(original_lines, key))
    print(pp(t/o), "of transcripts were transferred", o-t, "went missing")
quantify_transfer_percentage(r"final_versions\F_pennsylvanica_chr_20Feb2018_repeatmasker.gff3", 
                             "fraxinus_pennsylvanica_20Feb2018_repeatmasker.fasta.out.gff", 'similarity')

100.000000% of transcripts were transferred 0 went missing


In [94]:
quantify_transfer_percentage(r"final_versions\F_pennsylvanica_chr_BRAKER_RNASeqandEA_augustus_060618.hints.noevidenceMANUAL.gff3",
                             "BRAKER_RNASeqandEA_augustus_060618.hints.noevidenceMANUAL.gff", 'mRNA')

100.000000% of transcripts were transferred 0 went missing


In [95]:
quantify_transfer_percentage(r"final_versions\F_pennsylvanica_chr_BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL.gff3",
                             "BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL.gff", 'mRNA')

100.000000% of transcripts were transferred 0 went missing


## What are the 1,044 left out?

In [96]:
discarded = combine_lifted_and_non(
lifted = r"flo_pennsylvanica\run_simplified\scrubbed_BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL\lifted.gff3",
unlifted = r"flo_pennsylvanica\run_simplified\scrubbed_BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL\unlifted.gff3")

Output 4990 extra lines
Discarded 0 lines from chromosomes


In [73]:
from collections import Counter

Counter([l.split('\t')[0] for l in discarded])

Counter({'Scaffold_122': 52,
         'Scaffold_132': 28,
         'Scaffold_19': 28,
         'Scaffold_2': 24,
         'Scaffold_2026': 26,
         'Scaffold_222': 54,
         'Scaffold_243835': 54,
         'Scaffold_243836': 32,
         'Scaffold_243837': 38,
         'Scaffold_243839': 44,
         'Scaffold_243841': 26,
         'Scaffold_243842': 53,
         'Scaffold_243843': 50,
         'Scaffold_243844': 54,
         'Scaffold_243845': 7,
         'Scaffold_243846': 20,
         'Scaffold_243847': 50,
         'Scaffold_243848': 44,
         'Scaffold_243849': 49,
         'Scaffold_243850': 13,
         'Scaffold_243851': 54,
         'Scaffold_243852': 18,
         'Scaffold_291': 36,
         'Scaffold_3668': 33,
         'Scaffold_429': 10,
         'Scaffold_509': 26,
         'Scaffold_6': 26,
         'Scaffold_695': 39,
         'Scaffold_84': 64})